spark streaming

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import threading
import time
import socket

In [ ]:
#step1: create spark context
sc=SparkContext("local[2]","InbuiltDataSetStreaming")
ssc=StreamingContext(sc,3)

/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [ ]:
#step 2: streaming source socket
lines= ssc.socketTextStream("localhost",9999)

In [ ]:
#step3: processing logic
words=lines.flatMap(lambda line:line.split(" "))
pairs=words.map(lambda word:(word,1))
word_counts=pairs.reduceByKey(lambda a,b:a+b)

word_counts.pprint()

In [ ]:
word_counts.saveAsTextFiles("stream_output/wordcount")

In [ ]:
#step 4:start streaming job
def start_streaming():
  ssc.start()
  ssc.awaitTermination(60)
  ssc.stop(stopSparkContext=True,stopGraceFully=True)

In [ ]:
#step5: simulate stream with inbuilt dataset
def send_data():
  from sklearn.datasets import fetch_20newsgroups
  dataset=fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'))

  data=dataset.data[:50]
  server=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
  server.bind(('localhost',9999))
  server.listen(1)
  print("socket server started on port 9999.....")
  conn,addr=server.accept()

  for doc in data:
    line=doc.split("\n")[0]
    if line.strip():
      conn.send(line+"\n").encode("utf-8")
      time.sleep(2)

  conn.close()
  server.close()

In [ ]:
#step 6:run everything
threading.Thread(target=send_data).start
start_streaming()

-------------------------------------------
Time: 2025-09-27 07:52:18
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:21
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:24
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:27
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:30
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:33
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:36
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:39
-------------------------------------------

-------------------------------------------
Time: 2025-09-27 07:52:42
----------